In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

folder_path2 = "../processing_2"


## SMTHG on test train

In [ ]:
test_train = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_1/train_test.csv", index_col=False)

In [ ]:
test_train

In [ ]:
group_columns = ['Variete', 'Campagne', 'CodeTracabilite', 'Num Parcelle']
test_train['Recolte'] = test_train.groupby(group_columns).cumcount()

In [ ]:
test_train

In [ ]:
filtered_df = test_train[
    (test_train['Campagne'] == '21/22') &
    (test_train['Region'] == 'SOUSS') &
    (test_train['Site'] == 'Ouled Abbou') &
    (test_train['Famille'] == 'SPA') &
    (test_train['Variete'] == 'SPA4') &
    (test_train['Num Parcelle'] == '21071') &
    (test_train['CodeTracabilite'] == 'TR21071')
]

# Show the rows that match the criteria
filtered_df

In [ ]:
# Ensure the folder exists
if not os.path.exists(folder_path2):
    os.makedirs(folder_path2)

# Specify the full file path (folder path + file name)
file_path = os.path.join(folder_path2, 'train_test_perRecolte.csv')

# Save the DataFrame to a CSV file
test_train.to_csv(file_path, index=False)

## Merge it with irrigation

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../processing_2/train_test_perRecolte.csv")
df_irrig = pd.read_csv("../processing_1/combined_irrigation.csv")

df_irrig = df_irrig.rename(columns={'Code Traçabilité': 'CodeTracabilite'})

C:\Users\Oussama.ENNAJJAR\AppData\Local\Temp\ipykernel_14692\3109397961.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_irrig = pd.read_csv("../processing_1/combined_irrigation.csv")


In [3]:
df_irrig.isnull().sum()

Campagne           0
Region             0
Site               0
Famille            0
Variete            0
Num Parcelle       0
CodeTracabilite    0
Date               0
Qté eau (m3)       0
dtype: int64

In [4]:
# Convert 'Date' columns to datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
df_irrig['Date'] = pd.to_datetime(df_irrig['Date'], dayfirst=True, errors='coerce')

# Clean the 'Qté eau (m3)' column
df_irrig['Qté eau (m3)'] = df_irrig['Qté eau (m3)'].str.replace(',', '.')
df_irrig['Qté eau (m3)'] = df_irrig['Qté eau (m3)'].str.replace(r'[^\d.-]', '', regex=True).astype(float)


# # Sort both dataframes by necessary columns and 'Date'
# df= df.sort_values(by=['Campagne', 'Variete', 'Num Parcelle', 'CodeTracabilite', 'Date'])
# df_irrig = df_irrig.sort_values(by=['Campagne', 'Variete', 'Num Parcelle', 'CodeTracabilite', 'Date'])

C:\Users\Oussama.ENNAJJAR\AppData\Local\Temp\ipykernel_14692\1914679177.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_irrig['Date'] = pd.to_datetime(df_irrig['Date'], dayfirst=True, errors='coerce')


In [5]:
len(df_irrig[df_irrig["Qté eau (m3)"] != 0]) == len(df_irrig)

True

In [6]:
df.columns

Index(['Campagne', 'Region', 'Site', 'Famille', 'Variete', 'Num Parcelle',
       'CodeTracabilite', 'Date', 'Tonnage', 'is_train', 'Recolte'],
      dtype='object')

In [7]:
len(df[
    (df['Site'] == 'Rabat')&
    (df['is_train'] == False)
    ])

7

CORRECTIONS !

In [8]:
def remove_after_dot(s):
    dot_position = s.find('.')
    if dot_position == -1:
        return s
    return s[:dot_position]

df_irrig['Num Parcelle'] = df_irrig['Num Parcelle'].astype(str)
df['Num Parcelle'] = df['Num Parcelle'].astype(str)

df_irrig['Num Parcelle'] = df_irrig['Num Parcelle'].apply(remove_after_dot)

df['Num Parcelle'] = df['Num Parcelle'].str.strip()
df_irrig['Num Parcelle'] = df_irrig['Num Parcelle'].str.strip()

df['CodeTracabilite'] = df['CodeTracabilite'].str.strip()
df_irrig['CodeTracabilite'] = df_irrig['CodeTracabilite'].str.strip()

In [9]:
df['Site'].replace("Bengu?rir", "Benguérir", inplace=True)

In [10]:
df_irrig["Num Parcelle"].unique()
# ▶️ Num Parcelle existant à train_test dataset, not irrigation: ['13130' '11112' '15070' '20093' '12251' '15040' '15080' '15090' '15100'
#  '15120' '19999' '23000' '10070' '11222' '40131' '22014' '12252' '22012'
#  '12022' '12122'], soit: 20

array(['11011', '12010', '12020', '12050', '13010', '15060', '14010',
       '13030', '18030', '12030', '12040', '17010', '17020', '17030',
       '17040', '17050', '17060', '17070', '22020', '22030', '22040',
       '11010', '11020', '11030', '23020', '23040', '23050', '23060',
       '23090', '23120', '22010', '19010', '13020', '23010', '23030',
       '15010', '11240', '11130', '11212', '11211', '11140', '11160',
       '11170', '14020', '18010', '16010', '16060', '21010', '21020',
       '16020', '16030', '16050', '16040', '18020', '10001', '10003',
       '10007', '10008', '10009', '10010', '10011', '10012', '10031',
       '10033', '10034', '10035', '20001', '20002', '20003', '20004',
       '20005', '20006', '20007', '20008', '10005', '10036', '13051',
       '13061', '13070', '13080', '13090', '13100', '13110', '13120',
       '13170', '15070', '13021', '13031', '13041', '13130', '11050',
       '11112', '12090', '12150', '13150', '11222', '11221', '11223',
       '12222', '160

In [24]:
df_irrig["CodeTracabilite"].unique()
# ▶️ Code tracabilité existant à train_test dataset, not irrigation: ['TR12251' 'TR22014' 'TR12252' 'TR22012' 'TR12022' 'TR12122'], soit: 6

array(['TR11011', 'TR12010', 'TR12020', 'TR12050', 'TR13010', 'TR15060',
       'TR14010', 'TR11130', 'TR11140', 'TR11120', 'TR11090', 'TR11100',
       'TR11110', 'TR23070', 'TR23080', 'TR11070', 'TR11060', 'TR11080',
       'TR11030', 'TR11040', 'TR11010', 'TR11020', 'TR23020', 'TR23040',
       'TR23050', 'TR23060', 'TR11050', 'TR11240', 'TR11212', 'TR11211',
       'TR11160', 'TR11170', 'TR14000', 'TR17010', 'TR18010', 'TR19010',
       'TR13000', 'TR13030', 'TR11000', 'TR12000', 'TR16000', 'TR21010',
       'TR21020', 'TR15010', 'TR16020', 'TR16030', 'TR16050', 'TR16040',
       'TR18020', 'TR22050', 'TR22141', 'TR22220', 'TR21040', 'TR30099',
       'TR30100', 'TR30101', 'TR22130', 'TR22142', 'TR31040', 'TR22031',
       'TR14020', 'TR14030', 'TR14040', 'TR12030', 'TR13020', 'TR13040',
       'TR32010', 'TR22034', 'TR13051', 'TR13061', 'TR13070', 'TR13080',
       'TR13090', 'TR13100', 'TR13110', 'TR13120', 'TR13170', 'TR11003',
       'TR11402', 'TR11302', 'TR11105', 'TR11203', 

In [12]:
unique_traca_df = df[~df['CodeTracabilite'].isin(df_irrig['CodeTracabilite'])]['CodeTracabilite'].unique()
unique_parcelle_df = df[~df['Num Parcelle'].isin(df_irrig['Num Parcelle'])]['Num Parcelle'].unique()
unique_region_df = df[~df['Region'].isin(df_irrig['Region'])]['Region'].unique()
unique_site_df = df[~df['Site'].isin(df_irrig['Site'])]['Site'].unique()
unique_variete_df = df[~df['Variete'].isin(df_irrig['Variete'])]['Variete'].unique()
unique_famille_df = df[~df['Famille'].isin(df_irrig['Famille'])]['Famille'].unique()



print(f'''
    ▶️ Code tracabilité existant à train_test dataset, not irrigation: {unique_traca_df}, soit: {len(unique_traca_df)}
    ▶️ Num Parcelle existant à train_test dataset, not irrigation: {unique_parcelle_df}, soit: {len(unique_parcelle_df)}
    ▶️ Region existant à train_test dataset, not irrigation: {unique_region_df}, soit: {len(unique_region_df)}
    ▶️ Site existant à train_test dataset, not irrigation: {unique_site_df}, soit: {len(unique_site_df)}
    ▶️ Variete existant à train_test dataset, not irrigation: {unique_variete_df}, soit: {len(unique_variete_df)}
    ▶️ Famille existant à train_test dataset, not irrigation: {unique_famille_df}, soit: {len(unique_famille_df)}

''')


    ▶️ Code tracabilité existant à train_test dataset, not irrigation: ['TR12251' 'TR22014' 'TR12252' 'TR22012' 'TR12022' 'TR12122'], soit: 6
    ▶️ Num Parcelle existant à train_test dataset, not irrigation: ['20093' '12251' '22014' '12252' '22012' '12022' '12122'], soit: 7
    ▶️ Region existant à train_test dataset, not irrigation: [], soit: 0
    ▶️ Site existant à train_test dataset, not irrigation: ['Rabat'], soit: 1
    ▶️ Variete existant à train_test dataset, not irrigation: [], soit: 0
    ▶️ Famille existant à train_test dataset, not irrigation: [], soit: 0




In [13]:
df['Region'].unique()

array(['GHARB', 'HAOUZ', 'SOUSS', 'TADLA', 'ORIENTAL'], dtype=object)

In [16]:
def get_irrigation_qte(row):
    # Filter the DataFrame based on all the conditions
    irrigation_qte = df_irrig[
        (df_irrig['Date'] <= row['Date']) &
        (df_irrig['Campagne'] == row['Campagne']) &
        (df_irrig['Region'] == row['Region']) &
        (df_irrig['Site'] == row['Site']) &
        (df_irrig['Famille'] == row['Famille']) &
        (df_irrig['Variete'] == row['Variete']) &
        (df_irrig['Num Parcelle'] == row['Num Parcelle']) &
        (df_irrig['CodeTracabilite'] == row['CodeTracabilite'])
    ][['Qté eau (m3)']]

    irrigation_cumulee = irrigation_qte.sum().iloc[0]
    irrigation_cumulee
    # Use .iloc[0] for positional indexing
    if irrigation_cumulee != 0:
        return irrigation_cumulee
    else:
        # print(row)
        # print("➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖")
        return irrigation_cumulee

In [17]:
# df[df['Num Parcelle'].astype(str).str.startswith('23010')]

In [18]:
df['irrigation_cumulee'] = df.apply(get_irrigation_qte, axis=1)

In [19]:
# r = len(df[df['Site'] == 'Rabat'])
# codes = len(df[df['CodeTracabilite'].isin(['TR12251', 'TR12252', 'TR22012', 'TR22014', 'TR12022', 'TR12122'])])
# nums = len(df[df['Num Parcelle'].isin(['40131', '10070', '13130', '11112', '11222', '12251', '12252', '20093', 
#                        '15070', '15040', '15080', '15090', '15100', '15120', '19999', '23000', 
#                        '22012', '22014', '12022', '12122'])])

# print(f'''
#     ▶️ df with site rabat: {r},
#     ▶️ df with the CodeTraca that doesn't exist in irrigation: {codes},
#     ▶️ df with the Num Parcelle that doesn't exist in irrigation: {nums},
# ''')

In [20]:
len(df[df['Site'] == 'Rabat']), len(df_irrig[df_irrig['Site'] == 'Rabat'])

(21, 0)

In [21]:
len(df[(df['irrigation_cumulee'] == 0)])

248

In [ ]:
# len(df[df['irrigation_cumulee'] == 0]) ====> 248

In [ ]:
# get_irrigation_qte(test_row[0], test_row[1], test_row[2], test_row[3], test_row[4], test_row[5], test_row[6], test_row[7])

In [ ]:
# # Iterate over df1 and sum irrigation for each period from the earliest available date to the end date
# for index, row in df1.iterrows():
#     mask = (
#         (df2['Campagne'] == row['Campagne']) &
#         (df2['Variete'] == row['Variete']) &
#         (df2['Num Parcelle'] == row['Num Parcelle']) &
#         (df2['CodeTracabilite'] == row['CodeTracabilite']) &
#         (df2['Date'] <= row['Date'])  # Using row['Date'] as the end date
#     )
#     df1.at[index, 'Summed_Irrigation'] = df2.loc[mask, 'Qté eau (m3)'].sum()

## Add NPK

In [25]:
df_npk = pd.read_csv("../processing_1\combined_Apports_NPK.csv", index_col=False)

# Clean the 'Qté eau (m3)' column
df_npk['N'] = df_npk['N'].str.replace(',', '.')
df_npk['N'] = df_npk['N'].str.replace(r'[^\d.-]', '', regex=True).astype(float)

df_npk['P'] = df_npk['P'].str.replace(',', '.')
df_npk['P'] = df_npk['P'].str.replace(r'[^\d.-]', '', regex=True).astype(float)

df_npk['K'] = df_npk['K'].str.replace(',', '.')
df_npk['K'] = df_npk['K'].str.replace(r'[^\d.-]', '', regex=True).astype(float)

df_npk['Date'] = pd.to_datetime(df_npk['Date'], dayfirst=True)

C:\Users\Oussama.ENNAJJAR\AppData\Local\Temp\ipykernel_14692\3181972718.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_npk = pd.read_csv("../processing_1\combined_Apports_NPK.csv", index_col=False)
C:\Users\Oussama.ENNAJJAR\AppData\Local\Temp\ipykernel_14692\3181972718.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_npk['Date'] = pd.to_datetime(df_npk['Date'], dayfirst=True)


In [26]:
# Iterate over df1 and sum NPK for each period
for index, row in df.iterrows():
    mask = (
        (df_npk['Campagne'] == row['Campagne']) &
        (df_npk['Variete'] == row['Variete']) &
        (df_npk['Num Parcelle'] == row['Num Parcelle']) &
        (df_npk['CodeTracabilite'] == row['CodeTracabilite']) &
        (df_npk['Date'] <= row['Date'])
    )
    df.at[index, 'N_cumulee'] = df_npk.loc[mask, 'N'].sum()
    df.at[index, 'P_cumulee'] = df_npk.loc[mask, 'P'].sum()
    df.at[index, 'K_cumulee'] = df_npk.loc[mask, 'K'].sum()

In [28]:
import os
# Ensure the folder exists
if not os.path.exists(folder_path2):
    os.makedirs(folder_path2)

# Specify the full file path (folder path + file name)
file_path = os.path.join(folder_path2, 'train_test_merged_new.csv')

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

NameError: name 'folder_path2' is not defined